## Carga del dataset

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True
)

## Tamaño del dataset y clases

In [ ]:
ds_info

## Dimensiones de las imágenes

In [ ]:
for img, label in ds_train.take(1):
    print(img.shape, label)


## Visualización de ejemplos

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
for i, (img, label) in enumerate(ds_train.take(9)):
    plt.subplot(3,3,i+1)
    plt.imshow(img)
    plt.title("Dog" if label == 1 else "Cat")
    plt.axis("off")
plt.show()


## Preprocesamiento necesario

In [ ]:
IMG_SIZE = 128

def preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label


In [ ]:
BATCH_SIZE = 32

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE)


## Cierre conceptual de la EDA

This exploratory analysis shows that Cats vs Dogs is a non-trivial image classification problem with high variability and complex spatial structure. Such characteristics make convolutional layers particularly suitable due to their ability to exploit local correlations and spatial hierarchies.